In [ ]:
%pwd
%cd ../..

In [ ]:
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
# import polars as pl

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.optimization import (
    run_deeplifting,
    run_differential_evolution,
    run_dual_annealing,
    run_ipopt,
    run_pygranso,
)
from deeplifting.utils import create_optimization_plot

In [ ]:
# fig = create_optimization_plot(problem_name, problem, results, colormap='autumn_r')

In [ ]:
# fig.savefig(
#     '/Users/ryandevera/data-science/umn_environments/Deeplifting'
#     f'/images/{problem_name}_surface_and_contour-autumn.png'
# )

In [ ]:
# Run all problems
problem_names = [
    # 'ackley',
    'bukin_n6',
    # 'cross_in_tray',
    # 'drop_wave',
    # 'eggholder',
    # 'griewank',
    # 'holder_table',
    # 'levy',
    # 'levy_n13',
    # 'rastrigin',
    # 'schaffer_n2',
    # 'schaffer_n4',
    # 'schwefel',
    # 'shubert',
    # 'ex8_1_1',
    # # 'ex8_6_2',  # Need to fix threw an error
    # 'kriging_peaks_red010',
    # 'kriging_peaks_red020',
    # 'kriging_peaks_red030',
    # 'kriging_peaks_red050',
    # 'kriging_peaks_red100',
    # 'kriging_peaks_red200',
    # 'kriging_peaks_red500',
    # 'mathopt6',
    # 'quantum',
    # 'rosenbrock',
    # 'cross_leg_table',
    # 'sine_envelope',
]

problem_performance = []
trials = 1

for problem_name in problem_names:
    problem = PROBLEMS_BY_NAME[problem_name]
    outputs = run_deeplifting(
        problem,
        trials=trials,
        input_size=1024,
        hidden_sizes=(768, 768, 768),
        activation='sine',
        output_activation='sine',
        agg_function='max',
    )
    results = outputs['final_results']

    # # get the global minimum and the number of hits
    # minimum_value = problem['global_minimum']
    # hit = np.array([np.abs(found_min - minimum_value) for _, _, found_min in results])
    # hits = np.where(hit <= 1e-4, 1, 0).mean()

    # # Create a performance dataframe
    # df = pl.DataFrame(
    #     {
    #         'problem_name': [problem_name] * trials,
    #         'algorithm': ['deeplifting'] * trials,
    #         'hits': hits,
    #     }
    # )
    # problem_performance.append(df)

    # Let's plot the values with UMN colors
    fig = create_optimization_plot(
        problem_name, problem, results, add_contour_plot=True, colormap='OrRd'
    )

In [ ]:
results

In [ ]:
# fig.subplots_adjust(left=0, right=2, bottom=0, top=2)
# fig.savefig(
#     '/Users/ryandevera/data-science/umn_environments/Deeplifting'
#     f'/images/ackley_introduction.png',
#     bbox_inches='tight',
#     pad_inches=-0.1,
# )

In [ ]:
# performance_df = pl.concat(problem_performance, how='vertical')

In [ ]:
# performance_df = (
#     performance_df.groupby(['problem_name'])
#     .agg(pl.col('hits').mean())
#     .rename({'hits': 'success_rate'})
#     .sort('problem_name')
# )

In [ ]:
# performance_df

In [ ]:
# problem_name = 'bukin_n6'
# problem = PROBLEMS_BY_NAME[problem_name]
# problem['bounds'] = [(-10.0001, -9.9999), (1.0001, 0.9999)]
# results = np.array([[-10, 1, 0, 0, 0]])

# create_optimization_plot(
#     problem_name, problem, results, add_contour_plot=True, colormap='OrRd_r'
# )

In [ ]:
[64] * 2

In [ ]:
from itertools import product

input_sizes = [32, 64, 128, 256, 512, 1024]

hidden_size_64 = (64,)
hidden_size_128 = (128,)
hidden_size_256 = (256,)
hidden_size_512 = (512,)
hidden_size_768 = (768,)
hidden_size_1024 = (1024,)
hidden_size_2048 = (2048,)

hidden_sizes = [
    # Hidden sizes of 64
    hidden_size_64 * 2,
    # hidden_size_64 * 3,
    # hidden_size_64 * 4,
    # hidden_size_64 * 5,

    # # Hidden sizes of 128
    # hidden_size_128 * 2,
    # hidden_size_128 * 3,
    # hidden_size_128 * 4,
    # hidden_size_128 * 5,

    # # Hidden sizes of 256
    # hidden_size_256 * 2,
    # hidden_size_256 * 3,
    # hidden_size_256 * 4,
    # hidden_size_256 * 5,

    # # Hidden sizes of 512
    # hidden_size_512 * 2,
    # hidden_size_512 * 3,
    # hidden_size_512 * 4,
    # hidden_size_512 * 5,

    # # Hidden sizes of 768
    # hidden_size_768 * 2,
    # hidden_size_768 * 3,
    # hidden_size_768 * 4,
    # hidden_size_768 * 5,

    # # Hidden sizes of 1024
    # hidden_size_1024 * 2,
    # hidden_size_1024 * 3,
    # hidden_size_1024 * 4,
    # hidden_size_1024 * 5,

    # # Hidden sizes of 2048
    # hidden_size_2048 * 2,
    # hidden_size_2048 * 3,
    # hidden_size_2048 * 4,    
]

hidden_activations = ['sine', 'relu', 'leaky_relu']
output_activations = ['sine', 'leaky_relu']
agg_functions = ['max', 'sum', 'average']

combinations = (input_sizes, hidden_sizes, hidden_activations, output_activations, agg_functions)

len(list(product(*combinations)))

In [2]:
%pwd
%cd ../..

/home/rydevera3/data-science/umn_research/Deeplifting


In [16]:
%ls ./results

results-0.parquet                             results-48.parquet
results-2023-08-03-0-bukin_n6-0.parquet       results-49.parquet
results-2023-08-03-0-griewank-0.parquet       results-4.parquet
results-2023-08-03-0-schwefel-0.parquet       results-50.parquet
results-2023-08-03-1-bukin_n6-0.parquet       results-51.parquet
results-2023-08-03-1-griewank-0.parquet       results-52.parquet
results-2023-08-03-1-schwefel-0.parquet       results-53.parquet
results-2023-08-03-2-ackley-0.parquet         results-54.parquet
results-2023-08-03-2-bukin_n6-0.parquet       results-55.parquet
results-2023-08-03-2-cross_in_tray-0.parquet  results-56.parquet
results-2023-08-03-2-drop_wave-0.parquet      results-57.parquet
results-2023-08-03-2-eggholder-0.parquet      results-58.parquet
results-2023-08-03-2-griewank-0.parquet       results-59.parquet
results-2023-08-03-2-holder_table-0.parquet   results-5.parquet
results-2023-08-03-2-levy-0.parquet           results-60.parquet
results-2023-08-03-2-levy_n

In [22]:
import glob
import pandas as pd

In [23]:
results_df = pd.read_parquet(glob.glob('./results/results-2023-08-03-2*')).sort_values('f')

In [24]:
results_df.head(20)

,x1,x2,f,algorithm,total_time,input_size,hidden_size,hidden_activation,output_activation,agg_function,problem_name,global_minimum
95,512.000000,404.231805,-959.640663,Deeplifting,314.987315,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407
94,512.000000,404.231805,-959.640663,Deeplifting,238.053344,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407
92,512.000000,404.231806,-959.640663,Deeplifting,356.769283,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407
98,512.000000,404.231804,-959.640663,Deeplifting,119.819386,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407
93,511.999999,404.235347,-959.640647,Deeplifting,93.476720,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407
96,484.914277,435.722950,-953.418080,Deeplifting,32.001387,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407
99,511.221500,407.263876,-940.770825,Deeplifting,80.321372,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407
97,493.466803,448.357400,-909.467257,Deeplifting,23.324687,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407
90,242.037696,256.378969,-540.426295,Deeplifting,11.020405,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407
91,442.181567,-24.216495,-443.497384,Deeplifting,18.938473,512,"(128, 128)",sine,leaky_relu,sum,eggholder,-959.6407


In [20]:
(
    results_df[['x1', 'x2', 'f', 'problem_name']]
    .sort_values(['problem_name', 'f'])
    .groupby('problem_name')
    .agg({'f': 'describe'})
)

f                                                          \
              count          mean           std           min           25%   
problem_name                                                                  
ackley         10.0 -8.253661e-08  1.534190e-11 -8.254838e-08 -8.254792e-08   
bukin_n6       10.0  3.508878e-03  3.813122e-03  2.763092e-05  3.347481e-05   
cross_in_tray  10.0 -2.062612e+00  1.046728e-15 -2.062612e+00 -2.062612e+00   
drop_wave      10.0 -1.000000e+00  5.412281e-13 -1.000000e+00 -1.000000e+00   
eggholder      10.0 -8.585783e+02  1.951981e+02 -9.596407e+02 -9.596407e+02   
griewank       10.0  7.396040e-04  2.338833e-03  0.000000e+00  1.942890e-16   
holder_table   10.0 -1.358249e+01  7.454140e+00 -1.920850e+01 -1.920850e+01   
levy           10.0  7.033748e-16  6.664427e-19  7.031297e-16  7.031299e-16   
levy_n13       10.0  1.140025e-15  3.408847e-15  6.332335e-18  6.332335e-18   
rastrigin      10.0  2.788880e-14  3.558679e-14  0.000000e+00  4.440892e-16   
schaffer_n2    10.0  2.886580e-15  5.196877e-15  0.000000e+00  0.000000e+00   
schaffer_n4    10.0  2.925786e-01  5.615322e-15  2.925786e-01  2.925786e-01   
schwefel       10.0  2.827191e+00  8.940281e+00  2.545513e-05  2.545513e-05   

                                                         
                        50%           75%           max  
problem_name                                             
ackley        -8.254281e-08 -8.253254e-08 -8.250284e-08  
bukin_n6       2.352494e-03  6.287540e-03  1.058207e-02  
cross_in_tray -2.062612e+00 -2.062612e+00 -2.062612e+00  
drop_wave     -1.000000e+00 -1.000000e+00 -1.000000e+00  
eggholder     -9.565294e+02 -9.172931e+02 -4.434974e+02  
griewank       1.759703e-14  1.180028e-12  7.396040e-03  
holder_table  -1.910750e+01 -5.996805e+00 -1.732972e+00  
levy           7.031305e-16  7.032387e-16  7.052637e-16  
levy_n13       6.332388e-18  1.080993e-17  1.083124e-14  
rastrigin      5.329071e-15  5.817569e-14  9.237056e-14  
schaffer_n2    2.220446e-16  2.109424e-15  1.465494e-14  
schaffer_n4    2.925786e-01  2.925786e-01  2.925786e-01  
schwefel       2.545514e-05  2.545515e-05  2.827168e+01

In [21]:
import numpy as np
np.abs(0.001 - 0) <= 1e-4

False